¡Hola!

Mi nombre es Marcos Torres y tengo el gusto de revisar tu código el día de hoy.

Cuando vea algo notable o algún asunto en el notebook, te dejaré un comentario o un hint. Se que encontraras la mejor respuesta para resolver todos los comentarios, de no ser así, no te preocupes en futuras iteraciones dejaré comentarios y pistas más específicos.

Encontrarás comentarios en verde, amarillo o rojo como los siguientes:

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Buen trabajo. ¡Lo hiciste muy bien!
</div>

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Nota. Se puede mejorar.
</div>

<div class="alert alert-block alert-danger">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Necesitas corregirlo. Este bloque indica que se requiere una correción. El trabajo no se acepta si tiene estos bloques.
</div>

Puedes responder a mis comentarios usando estos bloques:

<div class="alert alert-block alert-info">
<b>Respuesta del estudiante.</b> <a class="tocSkip"></a>
</div>

# Proyecto SQL

## Objetivo general del estudio  
Identificar tendencias en la publicación y recepción de libros para generar una propuesta de valor efectiva para un nuevo producto.  

## Objetivos específicos
- Determinar la cantidad de libros recientes.
- Analizar la popularidad de los libros según reseñas y calificación promedio.
- Identificar la editorial con mayor volumen de publicaciones.
- Determinar el autor mejor calificado.
- Comprender el comportamiento de los usuarios más activos en la plataforma.  

### Conectarse a la base de datos

In [7]:
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
db_config = {'user': 'practicum_student',        
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', 
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              
             'db': 'data-analyst-final-project-db'}          

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Bien hecho usaste una celda independiente para importar las librerias y otra para definir la configuración a la base de datos.
</div>

### Probar la conexión

In [13]:
query = "SELECT * FROM books LIMIT 5;"
df = pd.io.sql.read_sql(query, con=engine)
df.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

¡Muy bien! Se ha probado correctamente la conexión a la base de datos.
</div>

### Encontrar el número de libros publicados después del 1 de enero de 2000

In [15]:
query = "SELECT COUNT(*) AS libros_publicados FROM books WHERE publication_date > '2000-01-01';"
resultado = pd.io.sql.read_sql(query, con=engine)
resultado

,libros_publicados
0,819


819 libros fueron publicados después del 1 de enero de 2000, o sea que son la cantidad de títulos recientes en la base de datos y podría ser útil este dato para analizar tendencias editoriales.

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Bien contaste correctamente los libros filtrados
</div>

### Encontrar el número de reseñas de usuarios y la calificación promedio para cada libro

In [12]:
query = """
SELECT r.book_id, COUNT(r.review_id) AS total_reseñas, AVG(rt.rating) AS calificación_promedio
FROM reviews r
JOIN ratings rt ON r.book_id = rt.book_id
GROUP BY r.book_id;
"""
resultado = pd.io.sql.read_sql(query, con=engine)
resultado.head()

,book_id,total_reseñas,calificación_promedio
0,652,4,4.500000
1,273,4,4.500000
2,51,60,4.250000
3,951,4,4.000000
4,839,28,4.285714


Del cuadro anterior se puede observar que algunos libros tienen muchas reseñas (por ejemplo, el libro 51 con 60 reseñas), mientras que otros tienen pocas. Además, las calificaciones promedio oscilan entre 4.0 y 4.5, lo que indica que los libros suelen tener buena aceptación.

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Bien hecho, usaste correctamente el join y el group para encontrar las reseñas y las calificaciones.
</div>

### Identificar la editorial que ha publicado el mayor número de libros con más de 50 páginas

In [16]:
query = """
SELECT p.publisher, COUNT(b.book_id) AS cantidad_libros
FROM books b
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY cantidad_libros DESC
LIMIT 1;
"""
resultado = pd.io.sql.read_sql(query, con=engine)
resultado

,publisher,cantidad_libros
0,Penguin Books,42


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Bien identificaste correctamente que la editorial penguin books es la que tiene más libros largos.
</div>

Penguin Books es una editorial muy reconocida, lo que puede indicar que tiene una fuerte presencia en el mercado de libros de mayor volumen.

### Identificar al autor que tiene la más alta calificación promedio del libro

In [18]:
query = """
SELECT a.author, AVG(rt.rating) AS calificación_promedio
FROM books b
JOIN authors a ON b.author_id = a.author_id
JOIN ratings rt ON b.book_id = rt.book_id
GROUP BY a.author
HAVING COUNT(rt.rating_id) >= 50
ORDER BY calificación_promedio DESC
LIMIT 1;
"""
resultado = pd.io.sql.read_sql(query, con=engine)
resultado

,author,calificación_promedio
0,Diana Gabaldon,4.3


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Bien usaste correctamente el having para encontrar el autor buscado.
</div>

Gabaldon es una autora reconocida. Su presencia en la base de datos y su alta calificación sugieren que sus libros son muy apreciados.

### Encontrar el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros

In [20]:
query = """
SELECT AVG(reviews_por_usuario.total_reseñas) AS promedio_reseñas_texto
FROM (
    SELECT r.username, COUNT(r.review_id) AS total_reseñas
    FROM reviews r
    JOIN ratings rt ON r.username = rt.username
    GROUP BY r.username
    HAVING COUNT(rt.rating_id) > 50
) AS reviews_por_usuario;
"""
resultado = pd.io.sql.read_sql(query, con=engine)
resultado

,promedio_reseñas_texto
0,719.75625


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Bien, aplicaste correctamente el query anidado para calcular este promedio.
</div>

En promedio, los usuarios que han calificado más de 50 libros han escrito 719.76 reseñas de texto. Esto sugiere que los usuarios más activos no solo califican los libros, sino que también dejan comentarios detallados sobre ellos.

## Conclusiones

El análisis revela datos clave que pueden guiar el desarrollo de un nuevo producto en el sector editorial:

- Oportunidad en libros recientes: hay 819 títulos publicados después del 2000, lo que indica un mercado activo en contenido moderno. Un producto que destaque estos libros podría atraer lectores interesados en publicaciones actuales.

- Importancia de las reseñas y calificaciones: los libros con más reseñas mantienen calificaciones promedio entre 4.0 y 4.5, lo que sugiere que los usuarios confían en la retroalimentación de la comunidad. Un producto que aproveche las reseñas como guía de calidad podría mejorar la experiencia del usuario.

- Liderazgo de Penguin Books en publicaciones: Con 42 títulos, esta editorial domina el mercado de libros extensos. Un producto que colabore con editoriales reconocidas podría aumentar su credibilidad.

- Autores destacados en calidad: Diana Gabaldon es la autora con mejor calificación promedio (4.3), mostrando que ciertos escritores pueden impulsar el éxito de un libro. Un producto enfocado en obras de autores con alta reputación podría atraer más lectores.

- Usuarios activos y generación de contenido: Los usuarios que calificaron más de 50 libros han escrito en promedio 719.76 reseñas de texto, lo que confirma que la comunidad de lectores valora compartir sus opiniones. Un producto que incentive la participación activa podría fortalecer su impacto en el mercado.

En general, un nuevo producto enfocado en destacar libros recientes y mejor calificados, incentivar la participación de la comunidad, y colaborar con editoriales reconocidas y autores exitosos, puede ser una propuesta atractiva para el mercado editorial.

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

¡Muy buenas conclusiones! Basadas en todo lo visto a lo largo del proyecto y con recomendaciones muy útiles de acuerdo a lo que se analizó en las consultas de la base de datos.
</div>

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

¡Hola, Priscila!

Me gustó bastante tu proyecto, todo está muy ordenado y los queries se aplicaron correctamente. Siempre recuerda lo básico de SQL pues es de las herramientas que nunca falta en cualquier trabajo de Científico o Analista de datos. Te deseo mucho éxito en los siguientes sprints.

Saludos, Marcos.
</div>